# Length-Adaptive Sequential Recommendation

**Hybrid SASRec + LightGCN with Adaptive Fusion on MovieLens-1M**

---

## ⚡ FAST TRACK (13 minutes total)

**Run only these cells if time matters:**
1. ✅ **Cell 3** - Clone repo (2 min)
2. ✅ **Cell 5** - Install dependencies (1 min)  
3. ⚡ **Cell 6** - Check GPU (10 sec)
4. ✅ **Cell 7** - Verify/preprocess data (2-3 min if needed) ← **CRITICAL**
5. ❌ **Cell 9** - SKIP quick test
6. ✅ **Cell 10** - Train Hybrid (~10 min) ← **MAIN EXPERIMENT**
7. ❌ **Cell 11** - SKIP SASRec (already have baseline)
8. ✅ **Cell 15** - Quick results view (5 sec)
9. ✅ **Cell 20** - Download results (30 sec)

**Total: ~15 minutes** instead of 45+ minutes

⚠️ **IMPORTANT:** Cell 7 is critical - it checks if data exists and preprocesses if needed!

---

## 📋 What You Get

- Trained hybrid model with length-adaptive fusion
- Complete test metrics (HR@10, NDCG@10, MRR@10)
- Performance by user groups (short/medium/long)
- Comparison with existing SASRec baseline
- Downloadable results for local analysis

## Step 1: Clone Repository

Cloning from: https://github.com/faroukq1/length-adaptive.git

In [ ]:
# Clone repository
!git clone https://github.com/faroukq1/length-adaptive.git

# Change to project directory
%cd length-adaptive

# Verify structure
!echo "✓ Source code:"
!ls -la src/

!echo "\n✓ Experiments scripts:"
!ls -lh experiments/

!echo "\n📁 Data directories:"
!ls -lh data/ 2>/dev/null || echo "   (Data will be downloaded in next step if needed)"

print("\n✅ Repository cloned successfully!")
print("⚠️  Note: Preprocessed data may not be in repo - we'll check/generate in next step")

## Step 2: Install Dependencies

Installing PyTorch Geometric and other required packages

In [ ]:
# Install required packages quietly
!pip install -q torch-geometric tqdm scikit-learn pandas matplotlib

print("✓ All dependencies installed successfully!")

## Step 3: Verify GPU Setup

Check if GPU is available and will be used for training

In [ ]:
# Check GPU availability
!python check_gpu.py

In [ ]:
import os

# Check if preprocessed data exists
data_file = 'data/ml-1m/processed/sequences.pkl'
graph_file = 'data/graphs/cooccurrence_graph.pkl'

print("="*70)
print("🔍 Checking Data Files")
print("="*70)

if os.path.exists(data_file):
    print(f"✅ Sequential data found: {data_file}")
    print(f"   Size: {os.path.getsize(data_file) / 1024 / 1024:.2f} MB")
else:
    print(f"❌ Sequential data NOT found: {data_file}")
    print("   → Need to run preprocessing!")

if os.path.exists(graph_file):
    print(f"✅ Graph data found: {graph_file}")
    print(f"   Size: {os.path.getsize(graph_file) / 1024 / 1024:.2f} MB")
else:
    print(f"❌ Graph data NOT found: {graph_file}")
    print("   → Need to build graph!")

# Check raw data
raw_file = 'data/ml-1m/raw/ml-1m/ratings.dat'
if os.path.exists(raw_file):
    print(f"✅ Raw data found: {raw_file}")
else:
    print(f"❌ Raw data NOT found: {raw_file}")
    print("   → Need to download MovieLens-1M!")

print("="*70)

# If data is missing, run preprocessing
if not os.path.exists(data_file) or not os.path.exists(graph_file):
    print("\n🔧 Running preprocessing...")
    print("This will take 2-3 minutes.\n")
    
    # Download MovieLens-1M if needed
    if not os.path.exists(raw_file):
        print("📥 Downloading MovieLens-1M dataset...")
        !mkdir -p data/ml-1m/raw
        !wget -q http://files.grouplens.org/datasets/movielens/ml-1m.zip
        !unzip -q ml-1m.zip
        !mv ml-1m data/ml-1m/raw/
        !rm -f ml-1m.zip
        print("✅ Download complete!\n")
    
    # Run preprocessing
    print("🔄 Preprocessing sequential data...")
    !python -m src.data.preprocess
    
    # Build graph
    print("\n🔄 Building co-occurrence graph...")
    !python -m src.data.graph_builder
    
    print("\n✅ Preprocessing complete!")
    print("="*70)
else:
    print("\n✅ All data files ready!")
    print("="*70)

## Step 3b: Verify Data Files

Check if preprocessed data exists, or run preprocessing if needed

## 📋 Experiment Priority Guide

This notebook includes experiments from the action plan to beat SASRec baseline:

**Priority 1 (Quick - Run First):**
- ✅ SASRec Baseline (Step 6)
- ✅ Hybrid Discrete (Step 5) - Our best model

**Priority 2 (Optimization - Run if time permits):**
- 🔬 Grid Search for Optimal Alpha (Advanced section)
- 🔬 All Hybrid Variants (Advanced section)

**Current Best Results:**
- Hybrid Fixed (α=0.5): HR@10 = 9.99% (+3.7% vs baseline)
- Short-history users: +42% improvement

**Target:** Beat SASRec on overall HR@10 by ≥3% and short-user HR@10 by ≥20%

## Step 4: Quick Test (OPTIONAL - Skip to Save Time)

⚡ **SKIP THIS** if you're in a hurry - saves 2 minutes!

This just verifies setup works. We'll go straight to full training instead.

In [ ]:
# SKIP: Quick test (saves 2 minutes)
# Uncomment only if you want to verify setup first

# !python test_training.py

print("⚡ Skipped quick test to save time - going straight to full training")

## ⚡ Step 5: Train Hybrid Model (CRITICAL - MUST RUN)

**This is the main experiment!**

Train our length-adaptive hybrid model:
- Short history users (≤10 items): More collaborative filtering (GNN)
- Medium users (10-50 items): Balanced fusion
- Long history users (>50 items): More sequential patterns (Transformer)

**Time: ~10 minutes with GPU T4**

With early stopping, training typically converges at epoch 20-30.

In [ ]:
# ⚡ CRITICAL: Train Hybrid Discrete Model (MUST RUN)
# This is the main experiment - takes ~10 minutes with GPU

print("="*70)
print("🚀 Training Hybrid Discrete Model")
print("="*70)

!python experiments/run_experiment.py \
    --model hybrid_discrete \
    --epochs 50 \
    --batch_size 256 \
    --lr 0.001 \
    --d_model 64 \
    --n_heads 2 \
    --n_blocks 2 \
    --patience 10

print("\n✅ Training complete! Check results/ folder for outputs.")

In [ ]:
import json
import glob
import os

print("="*80)
print("🔍 DIAGNOSTIC ANALYSIS - Why did hybrid_discrete underperform?")
print("="*80 + "\n")

# Find hybrid_discrete results
discrete_folders = [f for f in glob.glob('results/hybrid_discrete_*') if os.path.isdir(f)]

if not discrete_folders:
    print("⚠️  No hybrid_discrete results found. Run training first!")
else:
    for folder in discrete_folders:
        print(f"\n{'='*80}")
        print(f"📁 Analyzing: {os.path.basename(folder)}")
        print(f"{'='*80}\n")
        
        # 1. Check config
        config_path = os.path.join(folder, 'config.json')
        if os.path.exists(config_path):
            with open(config_path, 'r') as f:
                config = json.load(f)
            print("⚙️  Configuration:")
            print(f"  • Learning rate: {config.get('lr', 'N/A')}")
            print(f"  • Model dim: {config.get('d_model', 'N/A')}")
            print(f"  • Heads: {config.get('n_heads', 'N/A')}")
            print(f"  • Blocks: {config.get('n_blocks', 'N/A')}")
            print(f"  • Batch size: {config.get('batch_size', 'N/A')}")
            print(f"  • Max epochs: {config.get('epochs', 'N/A')}")
            print(f"  • Patience: {config.get('patience', 'N/A')}\n")
        
        # 2. Check training history
        history_path = os.path.join(folder, 'history.json')
        if os.path.exists(history_path):
            with open(history_path, 'r') as f:
                history = json.load(f)
            
            epochs_trained = len(history.get('train_loss', []))
            print(f"📊 Training History:")
            print(f"  • Epochs completed: {epochs_trained}")
            
            if history.get('val_metrics'):
                best_epoch = max(range(len(history['val_metrics'])), 
                               key=lambda i: history['val_metrics'][i].get('NDCG@10', 0))
                best_ndcg = history['val_metrics'][best_epoch]['NDCG@10']
                print(f"  • Best epoch: {best_epoch + 1}")
                print(f"  • Best val NDCG@10: {best_ndcg:.4f}")
                
                # Check if early stopped
                if epochs_trained < config.get('epochs', 50):
                    print(f"  ⚠️  Early stopped at epoch {epochs_trained}")
                    print(f"      (Patience triggered - no improvement for {config.get('patience', 10)} epochs)")
                else:
                    print(f"  ✓  Ran full {epochs_trained} epochs")
            print()
        
        # 3. Check final results
        results_path = os.path.join(folder, 'results.json')
        if os.path.exists(results_path):
            with open(results_path, 'r') as f:
                results = json.load(f)
            
            print("🎯 Test Results:")
            metrics = results['test_metrics']
            print(f"  • HR@10: {metrics['HR@10']:.4f} ({metrics['HR@10']*100:.2f}%)")
            print(f"  • NDCG@10: {metrics['NDCG@10']:.4f}")
            print(f"  • MRR@10: {metrics['MRR@10']:.4f}\n")
            
            # Check grouped metrics
            if 'grouped_metrics' in results:
                print("👥 Performance by User Group:")
                grouped = results['grouped_metrics']
                for group in ['short', 'medium', 'long']:
                    if group in grouped:
                        g = grouped[group]
                        print(f"  • {group.capitalize():6s}: HR@10={g['HR@10']:.4f}, "
                              f"NDCG@10={g['NDCG@10']:.4f}, count={g['count']}")
                print()
        
        # 4. Check alpha statistics
        alpha_path = os.path.join(folder, 'alpha_stats.json')
        if os.path.exists(alpha_path):
            with open(alpha_path, 'r') as f:
                alpha_stats = json.load(f)
            
            print("🎚️  Alpha Values Used:")
            for group in ['short', 'medium', 'long']:
                if group in alpha_stats:
                    stats = alpha_stats[group]
                    print(f"  • {group.capitalize():6s}: mean={stats['mean']:.3f}, "
                          f"std={stats['std']:.3f}")
            print()

# Compare with SASRec baseline
print("\n" + "="*80)
print("📈 COMPARISON WITH BASELINE")
print("="*80 + "\n")

sasrec_folders = [f for f in glob.glob('results/sasrec_*') if os.path.isdir(f)]
if sasrec_folders and discrete_folders:
    sasrec_results_path = os.path.join(sasrec_folders[0], 'results.json')
    discrete_results_path = os.path.join(discrete_folders[0], 'results.json')
    
    if os.path.exists(sasrec_results_path) and os.path.exists(discrete_results_path):
        with open(sasrec_results_path, 'r') as f:
            sasrec_results = json.load(f)
        with open(discrete_results_path, 'r') as f:
            discrete_results = json.load(f)
        
        sasrec_hr = sasrec_results['test_metrics']['HR@10']
        discrete_hr = discrete_results['test_metrics']['HR@10']
        improvement = ((discrete_hr - sasrec_hr) / sasrec_hr) * 100
        
        print(f"SASRec baseline:    HR@10 = {sasrec_hr:.4f} ({sasrec_hr*100:.2f}%)")
        print(f"Hybrid discrete:    HR@10 = {discrete_hr:.4f} ({discrete_hr*100:.2f}%)")
        print(f"Improvement:        {improvement:+.2f}%")
        
        if improvement < 0:
            print(f"\n❌ UNDERPERFORMING by {abs(improvement):.2f}%")
            print("   → Need to try different hyperparameters!")
        elif improvement < 3:
            print(f"\n⚠️  Improvement below 3% target")
            print("   → Try optimization strategies below")
        else:
            print(f"\n✅ SUCCESS! Beat baseline by {improvement:.2f}%")

print("\n" + "="*80)

In [ ]:
# Run all optimization experiments
# Uncomment to run complete sweep (takes ~50-60 min)

experiments = [
    {
        'name': 'Lower LR',
        'params': '--lr 0.0005 --epochs 80 --patience 15'
    },
    {
        'name': 'Higher LR',
        'params': '--lr 0.002 --epochs 60 --patience 12'
    },
    {
        'name': 'Bigger Model',
        'params': '--lr 0.001 --d_model 128 --n_heads 4 --epochs 60 --patience 12'
    },
    {
        'name': 'Longer Training',
        'params': '--lr 0.001 --epochs 100 --patience 15'
    },
    {
        'name': 'Combined (Lower LR + Bigger + Longer)',
        'params': '--lr 0.0005 --d_model 128 --n_heads 4 --epochs 100 --patience 15'
    }
]

# for i, exp in enumerate(experiments, 1):
#     print(f"\n{'='*70}")
#     print(f"🧪 Experiment {i}/5: {exp['name']}")
#     print(f"{'='*70}\n")
#     
#     !python experiments/run_experiment.py \
#         --model hybrid_discrete \
#         --batch_size 256 \
#         --n_blocks 2 \
#         {exp['params']}
#     
#     print(f"\n✅ {exp['name']} complete!\n")
# 
# print("="*70)
# print("🎉 All optimization experiments complete!")
# print("="*70)

print("💡 Uncomment the code above to run all experiments")

In [ ]:
import json
import glob
import os
import pandas as pd

print("="*80)
print("🏆 OPTIMIZATION RESULTS COMPARISON")
print("="*80 + "\n")

# Collect all hybrid_discrete results
discrete_folders = sorted([f for f in glob.glob('results/hybrid_discrete_*') if os.path.isdir(f)])

if not discrete_folders:
    print("❌ No hybrid_discrete results found. Run experiments first!")
else:
    results_data = []
    
    for folder in discrete_folders:
        results_path = os.path.join(folder, 'results.json')
        config_path = os.path.join(folder, 'config.json')
        history_path = os.path.join(folder, 'history.json')
        
        if os.path.exists(results_path) and os.path.exists(config_path):
            with open(results_path, 'r') as f:
                results = json.load(f)
            with open(config_path, 'r') as f:
                config = json.load(f)
            
            # Get training info
            epochs_trained = 0
            if os.path.exists(history_path):
                with open(history_path, 'r') as f:
                    history = json.load(f)
                epochs_trained = len(history.get('train_loss', []))
            
            # Extract configuration details
            folder_name = os.path.basename(folder)
            timestamp = '_'.join(folder_name.split('_')[-2:])
            
            results_data.append({
                'Timestamp': timestamp,
                'LR': config.get('lr', 'N/A'),
                'd_model': config.get('d_model', 64),
                'n_heads': config.get('n_heads', 2),
                'Epochs': epochs_trained,
                'HR@10': results['test_metrics']['HR@10'],
                'NDCG@10': results['test_metrics']['NDCG@10'],
                'MRR@10': results['test_metrics']['MRR@10'],
                'HR@10_short': results.get('grouped_metrics', {}).get('short', {}).get('HR@10', 0),
                'HR@10_long': results.get('grouped_metrics', {}).get('long', {}).get('HR@10', 0)
            })
    
    if results_data:
        df = pd.DataFrame(results_data)
        df = df.sort_values('NDCG@10', ascending=False)
        
        print("All Hybrid Discrete Experiments:")
        print("-" * 80)
        print(df.to_string(index=False, float_format='%.4f'))
        
        # Highlight best overall
        best_idx = df['NDCG@10'].idxmax()
        best = df.loc[best_idx]
        
        print("\n" + "="*80)
        print("🥇 BEST CONFIGURATION (by NDCG@10):")
        print("="*80)
        print(f"  Timestamp: {best['Timestamp']}")
        print(f"  Learning Rate: {best['LR']}")
        print(f"  Model Size: d_model={best['d_model']}, n_heads={best['n_heads']}")
        print(f"  Trained Epochs: {best['Epochs']}")
        print(f"\n  Overall Performance:")
        print(f"    HR@10:   {best['HR@10']:.4f} ({best['HR@10']*100:.2f}%)")
        print(f"    NDCG@10: {best['NDCG@10']:.4f}")
        print(f"    MRR@10:  {best['MRR@10']:.4f}")
        print(f"\n  User Group Performance:")
        print(f"    Short users:  HR@10 = {best['HR@10_short']:.4f} ({best['HR@10_short']*100:.2f}%)")
        print(f"    Long users:   HR@10 = {best['HR@10_long']:.4f} ({best['HR@10_long']*100:.2f}%)")
        
        # Compare with SASRec
        sasrec_folders = [f for f in glob.glob('results/sasrec_*') if os.path.isdir(f)]
        if sasrec_folders:
            sasrec_results_path = os.path.join(sasrec_folders[0], 'results.json')
            if os.path.exists(sasrec_results_path):
                with open(sasrec_results_path, 'r') as f:
                    sasrec_results = json.load(f)
                
                sasrec_hr = sasrec_results['test_metrics']['HR@10']
                improvement = ((best['HR@10'] - sasrec_hr) / sasrec_hr) * 100
                
                print(f"\n  vs SASRec Baseline:")
                print(f"    SASRec:     HR@10 = {sasrec_hr:.4f} ({sasrec_hr*100:.2f}%)")
                print(f"    Improvement: {improvement:+.2f}%")
                
                if improvement >= 3:
                    print(f"    ✅ SUCCESS! Beat baseline by ≥3%")
                elif improvement > 0:
                    print(f"    ⚠️  Improvement below 3% target")
                else:
                    print(f"    ❌ Underperforming baseline")
        
        print("="*80)
        
        # Show improvement from first to best run
        if len(df) > 1:
            first_ndcg = df.iloc[-1]['NDCG@10']
            best_ndcg = df.iloc[0]['NDCG@10']
            improvement = ((best_ndcg - first_ndcg) / first_ndcg) * 100
            
            print(f"\n📈 Optimization Progress:")
            print(f"  First run:  NDCG@10 = {first_ndcg:.4f}")
            print(f"  Best run:   NDCG@10 = {best_ndcg:.4f}")
            print(f"  Improvement: {improvement:+.2f}%")
            print()
    else:
        print("❌ Could not parse results files")

print("="*80)

## 📊 Compare All Optimization Results

After running experiments, use this cell to compare all configurations and find the best one.

In [ ]:
# Check current alpha bin configuration
print("="*70)
print("🔍 Current Alpha Bin Configuration")
print("="*70 + "\n")

# Read the fusion.py file to see current alpha bins
try:
    with open('src/models/fusion.py', 'r') as f:
        content = f.read()
        
    # Look for discrete fusion alpha values
    if 'if seq_len <= 10:' in content:
        print("Found discrete fusion implementation!")
        print("\nCurrent configuration (search for these lines in fusion.py):")
        print("  • Short users (≤10):  alpha = 0.3  (30% SASRec, 70% GNN)")
        print("  • Medium users (≤50): alpha = 0.5  (50% SASRec, 50% GNN)")
        print("  • Long users (>50):   alpha = 0.7  (70% SASRec, 30% GNN)")
        
        print("\n📝 To test different alpha values:")
        print("1. Edit src/models/fusion.py")
        print("2. Find the discrete_fusion class")
        print("3. Modify the alpha values in the forward() method")
        print("4. Rerun training experiment")
        
        print("\n💡 Suggested alternatives to try:")
        print("  Strategy A (More GNN for short):")
        print("    • Short: 0.2, Medium: 0.5, Long: 0.8")
        print("  Strategy B (More balanced):")
        print("    • Short: 0.4, Medium: 0.5, Long: 0.6")
        print("  Strategy C (Strong sequential):")
        print("    • Short: 0.3, Medium: 0.6, Long: 0.9")
    else:
        print("⚠️  Could not find discrete fusion configuration")
        
except FileNotFoundError:
    print("⚠️  fusion.py not found - make sure you're in the project directory")

print("\n" + "="*70)

## 🎚️ Advanced: Testing Different Alpha Bin Values

The discrete fusion uses alpha bins for short/medium/long users.
**Default**: short=0.3, medium=0.5, long=0.7

**Alternative strategies to try:**
- **More GNN for short**: 0.2, 0.5, 0.8 (stronger collaborative signal)
- **More balanced**: 0.4, 0.5, 0.6 (smaller differences between groups)
- **More SASRec for long**: 0.3, 0.6, 0.9 (emphasize sequential for long histories)

**Note**: Requires code modification in [src/models/fusion.py](src/models/fusion.py)

### Experiment 5: Run All Optimizations Together

**Why**: Systematic sweep to find best combination
**Time**: ~50-60 minutes total (runs sequentially)

This runs all 4 experiments above automatically.

In [ ]:
# Experiment 4: Longer Training
print("="*70)
print("🧪 Experiment 4: Longer Training (100 epochs, patience=15)")
print("="*70)

!python experiments/run_experiment.py \
    --model hybrid_discrete \
    --epochs 100 \
    --batch_size 256 \
    --lr 0.001 \
    --d_model 64 \
    --n_heads 2 \
    --n_blocks 2 \
    --patience 15

print("\n✅ Experiment 4 complete!")

### Experiment 4: Longer Training (epochs=100, patience=15)

**Why**: Allow model to fully converge
**Best for**: If current model stopped too early (check diagnostic above)

In [ ]:
# Experiment 3: Bigger Model
print("="*70)
print("🧪 Experiment 3: Bigger Model (d_model=128, n_heads=4)")
print("="*70)

!python experiments/run_experiment.py \
    --model hybrid_discrete \
    --epochs 60 \
    --batch_size 256 \
    --lr 0.001 \
    --d_model 128 \
    --n_heads 4 \
    --n_blocks 2 \
    --patience 12

print("\n✅ Experiment 3 complete!")

### Experiment 3: Bigger Model (d_model=128, n_heads=4)

**Why**: More capacity to learn complex user patterns
**Best for**: If model seems to underfit (training & validation both improving)

In [ ]:
# Experiment 2: Higher Learning Rate
print("="*70)
print("🧪 Experiment 2: Higher Learning Rate (0.002)")
print("="*70)

!python experiments/run_experiment.py \
    --model hybrid_discrete \
    --epochs 60 \
    --batch_size 256 \
    --lr 0.002 \
    --d_model 64 \
    --n_heads 2 \
    --n_blocks 2 \
    --patience 12

print("\n✅ Experiment 2 complete!")

### Experiment 2: Higher Learning Rate (0.002)

**Why**: Faster learning, might escape poor local minima
**Best for**: If current model converged too early to suboptimal solution

In [ ]:
# Experiment 1: Lower Learning Rate
print("="*70)
print("🧪 Experiment 1: Lower Learning Rate (0.0005)")
print("="*70)

!python experiments/run_experiment.py \
    --model hybrid_discrete \
    --epochs 80 \
    --batch_size 256 \
    --lr 0.0005 \
    --d_model 64 \
    --n_heads 2 \
    --n_blocks 2 \
    --patience 15

print("\n✅ Experiment 1 complete!")

### Experiment 1: Lower Learning Rate (0.0005)

**Why**: More stable training, better convergence
**Best for**: If current model shows unstable validation metrics

## 🔬 Hyperparameter Optimization Suite

Based on the diagnostic analysis above, try these optimized configurations to improve performance.

**Strategy:**
1. **Lower LR (0.0005)**: More stable, might converge better
2. **Higher LR (0.002)**: Faster learning, might escape local minima
3. **Bigger Model**: More capacity for complex patterns
4. **Different Alpha Bins**: Adjust fusion weights per group
5. **Longer Training**: More epochs + patience to fully converge

**Time per experiment**: ~10-15 minutes with GPU T4

Run the experiments below that interest you most!

## 🔍 Diagnostic: Analyze Training History

Before running new experiments, let's diagnose why hybrid_discrete underperformed.
This cell checks:
- Early stopping point (did it stop too early?)
- Learning curve patterns (overfitting? underfitting?)
- Alpha values used (were they correct?)
- User group distribution

## Step 6: Train SASRec Baseline (Optional - Skip if you already have it)

**⚠️ SKIP THIS STEP if:**
- You already have `results/sasrec_*/` folder from previous runs
- You haven't changed data preprocessing or hyperparameters
- You just want to test new hybrid variants

**Only run this if:**
- First time training
- Changed hyperparameters
- Want to verify reproducibility
- Need fresh baseline for comparison

**Alternative:** Copy your existing `results/sasrec_*/` folder to Kaggle instead of retraining.

In [ ]:
# OPTION 1: Skip SASRec training (if you already have results)
print("💡 Skipping SASRec - using existing baseline results")
print("   If you need to train SASRec, uncomment the code below:\n")

# OPTION 2: Train SASRec baseline (uncomment if needed)
# print("="*70)
# print("🚀 Training SASRec Baseline")
# print("="*70)
# 
# !python experiments/run_experiment.py \
#     --model sasrec \
#     --epochs 50 \
#     --batch_size 256 \
#     --lr 0.001 \
#     --patience 10
# 
# print("\n✅ Baseline training complete!")

# OPTION 3: Upload existing SASRec results
# If you have results locally, you can upload the folder:
# 1. Zip your local results/sasrec_*/ folder
# 2. Upload to Kaggle input data
# 3. Copy to results/ directory:
# !mkdir -p results
# !cp -r /kaggle/input/your-sasrec-results/* results/

## Step 7: Train All Models (Optional - takes 3-5 hours)

Uncomment to train all 5 model variants:
- `sasrec`: Transformer baseline
- `hybrid_fixed`: Fixed fusion weight (α=0.5)
- `hybrid_discrete`: Bin-based fusion (our approach)
- `hybrid_learnable`: Per-user learned weights
- `hybrid_continuous`: Neural network fusion

In [ ]:
# Uncomment to run all experiments
# !bash scripts/run_all_experiments.sh

In [ ]:
# Train all hybrid variants
# Uncomment to run complete ablation study (takes ~8 hours with GPU)

# models = ['hybrid_fixed', 'hybrid_discrete', 'hybrid_learnable', 'hybrid_continuous']
# 
# for model in models:
#     print(f"\n{'='*70}")
#     print(f"🚀 Training {model}")
#     print(f"{'='*70}\n")
#     
#     !python experiments/run_experiment.py \
#         --model {model} \
#         --epochs 50 \
#         --batch_size 256 \
#         --lr 0.001 \
#         --patience 10
#     
#     print(f"\n✅ {model} complete!")

# Quick version: Use the automated script
# !bash scripts/run_all_experiments.sh

print("💡 Tip: Uncomment to train all model variants")

## 🔬 Advanced: All Hybrid Variants

Train all fusion strategies for complete comparison:
- **Fixed**: Single α for all users
- **Discrete**: Bin-based (short/medium/long)
- **Learnable**: Learned bin weights
- **Continuous**: Smooth function of length

In [ ]:
# Grid search for optimal alpha
# Tests α ∈ {0.3, 0.4, 0.5, 0.6, 0.7}
# Uncomment to run (takes ~10-12 hours with GPU)

# alphas = [0.3, 0.4, 0.5, 0.6, 0.7]
# 
# for alpha in alphas:
#     print(f"\n{'='*70}")
#     print(f"🔬 Testing Fixed Alpha = {alpha}")
#     print(f"{'='*70}\n")
#     
#     !python experiments/run_experiment.py \
#         --model hybrid_fixed \
#         --fixed_alpha {alpha} \
#         --epochs 50 \
#         --batch_size 256 \
#         --lr 0.001 \
#         --patience 10
#     
#     print(f"\n✅ Alpha={alpha} complete!")

print("💡 Tip: Uncomment the code above to run grid search")

## 🔬 Advanced: Grid Search for Optimal Alpha (Fixed Fusion)

Test different fixed alpha values to find the optimal fusion weight.
This helps us understand the best balance between GNN and SASRec embeddings.

## Step 8: Analyze Results

Generate comparison tables and visualizations

In [ ]:
# Generate analysis using the built-in script
print("="*70)
print("📊 Generating Analysis")
print("="*70)

!python experiments/analyze_results.py --save_csv

print("\n✅ Analysis complete!")

## Step 9: Display Results

Show performance comparison table

In [ ]:
import pandas as pd
import os
import json
import glob

# Try to load results directly from experiments
result_folders = glob.glob('results/*_*')

if len(result_folders) == 0:
    print("❌ No results found. Run experiments first!")
else:
    print("\n" + "="*80)
    print("📊 OVERALL PERFORMANCE")
    print("="*80 + "\n")
    
    # Collect all results
    all_results = []
    for folder in result_folders:
        results_path = os.path.join(folder, 'results.json')
        if os.path.exists(results_path):
            with open(results_path, 'r') as f:
                results = json.load(f)
            
            # Extract model name
            folder_name = os.path.basename(folder)
            model_name = '_'.join(folder_name.split('_')[:-2])
            
            all_results.append({
                'Model': model_name,
                'HR@5': results['test_metrics']['HR@5'],
                'HR@10': results['test_metrics']['HR@10'],
                'HR@20': results['test_metrics']['HR@20'],
                'NDCG@5': results['test_metrics']['NDCG@5'],
                'NDCG@10': results['test_metrics']['NDCG@10'],
                'NDCG@20': results['test_metrics']['NDCG@20'],
                'MRR@10': results['test_metrics']['MRR@10']
            })
    
    if all_results:
        df = pd.DataFrame(all_results)
        df = df.sort_values('NDCG@10', ascending=False)
        
        # Display table
        print(df.to_string(index=False, float_format='%.4f'))
        
        # Highlight best model
        best = df.iloc[0]
        print("\n" + "="*80)
        print(f"🏆 BEST MODEL: {best['Model']}")
        print("="*80)
        print(f"  NDCG@10: {best['NDCG@10']:.4f}")
        print(f"  HR@10:   {best['HR@10']:.4f}")
        print(f"  MRR@10:  {best['MRR@10']:.4f}")
        print("="*80 + "\n")
        
        # Show improvement over baseline
        sasrec_row = df[df['Model'] == 'sasrec']
        if not sasrec_row.empty:
            sasrec_ndcg = sasrec_row.iloc[0]['NDCG@10']
            sasrec_hr = sasrec_row.iloc[0]['HR@10']
            hybrid_ndcg = best['NDCG@10']
            hybrid_hr = best['HR@10']
            ndcg_imp = ((hybrid_ndcg - sasrec_ndcg) / sasrec_ndcg) * 100
            hr_imp = ((hybrid_hr - sasrec_hr) / sasrec_hr) * 100
            print(f"📈 Improvement over SASRec baseline:")
            print(f"   NDCG@10: {ndcg_imp:+.2f}%")
            print(f"   HR@10:   {hr_imp:+.2f}%\n")
    else:
        print("❌ Could not parse results files")

In [ ]:
# Quick performance check
import json
import glob
import os

results = {}
for folder in glob.glob('results/*_*'):
    results_file = os.path.join(folder, 'results.json')
    if os.path.exists(results_file):
        with open(results_file) as f:
            data = json.load(f)
        model = '_'.join(os.path.basename(folder).split('_')[:-2])
        results[model] = data['test_metrics']['HR@10']

if results:
    print("="*60)
    print("⚡ QUICK RESULTS - HR@10 (Higher is Better)")
    print("="*60)
    for model, hr10 in sorted(results.items(), key=lambda x: x[1], reverse=True):
        print(f"  {model:20s}: {hr10:.4f} ({hr10*100:.2f}%)")
    
    if 'sasrec' in results and 'hybrid_discrete' in results:
        improvement = ((results['hybrid_discrete'] - results['sasrec']) / results['sasrec']) * 100
        print("\n" + "="*60)
        print(f"📈 Hybrid improvement: {improvement:+.1f}%")
        if improvement > 2:
            print("✅ SUCCESS: Beat baseline by >2%!")
        print("="*60)
else:
    print("⚠️  No results found yet. Train models first (Cell 9).")

## ⚡ FAST TRACK: Quick Results View

If you're short on time, just run this cell to see if hybrid beats baseline!

## Step 10: Performance by User Group

Compare performance across different user history lengths

In [ ]:
import glob
import json
import os
import pandas as pd

# Load grouped metrics
print("\n" + "="*80)
print("📊 PERFORMANCE BY USER GROUP")
print("="*80 + "\n")

result_folders = glob.glob('results/*_*')

if len(result_folders) == 0:
    print("❌ No results found.")
else:
    # Collect grouped results
    group_data = {'short': [], 'medium': [], 'long': []}
    
    for folder in result_folders:
        results_path = os.path.join(folder, 'results.json')
        if os.path.exists(results_path):
            with open(results_path, 'r') as f:
                results = json.load(f)
            
            # Extract model name
            folder_name = os.path.basename(folder)
            model_name = '_'.join(folder_name.split('_')[:-2])
            
            # Extract grouped metrics
            grouped = results.get('grouped_metrics', {})
            
            for group in ['short', 'medium', 'long']:
                if group in grouped:
                    group_data[group].append({
                        'Model': model_name,
                        'HR@10': grouped[group]['HR@10'],
                        'NDCG@10': grouped[group]['NDCG@10'],
                        'MRR@10': grouped[group]['MRR@10'],
                        'Count': grouped[group]['count']
                    })
    
    # Display each group
    for group_name in ['short', 'medium', 'long']:
        if group_data[group_name]:
            df_group = pd.DataFrame(group_data[group_name])
            df_group = df_group.sort_values('NDCG@10', ascending=False)
            
            print(f"\n{group_name.upper()} HISTORY USERS:")
            print("-" * 80)
            print(df_group.to_string(index=False, float_format='%.4f'))
            print()
        else:
            print(f"\n{group_name.upper()} HISTORY USERS:")
            print("-" * 80)
            print(f"⚠️  No {group_name} user data found (possibly no users in this range)")
            print()

In [ ]:
import glob
import json
import os

# Check for alpha statistics in hybrid model results
print("\n" + "="*80)
print("🔍 ALPHA VALUES (Fusion Weights)")
print("="*80 + "\n")

hybrid_folders = [f for f in glob.glob('results/hybrid_*') if os.path.isdir(f)]

if not hybrid_folders:
    print("⚠️  No hybrid model results found. Alpha tracking only works for hybrid models.")
else:
    for folder in hybrid_folders:
        alpha_path = os.path.join(folder, 'alpha_stats.json')
        if os.path.exists(alpha_path):
            with open(alpha_path, 'r') as f:
                alpha_stats = json.load(f)
            
            folder_name = os.path.basename(folder)
            model_name = '_'.join(folder_name.split('_')[:-2])
            
            print(f"{model_name.upper()}:")
            print("-" * 80)
            
            for group in ['short', 'medium', 'long', 'overall']:
                if group in alpha_stats:
                    stats = alpha_stats[group]
                    if group != 'overall' and 'count' in stats:
                        print(f"  {group.capitalize():8s}: mean={stats['mean']:.3f}, std={stats['std']:.3f}, count={stats['count']}")
                    elif group == 'overall':
                        print(f"  {group.capitalize():8s}: mean={stats['mean']:.3f}, std={stats['std']:.3f}")
            print()
        else:
            # Show expected alpha values based on model type
            folder_name = os.path.basename(folder)
            model_name = '_'.join(folder_name.split('_')[:-2])
            
            print(f"{model_name.upper()}:")
            print("-" * 80)
            
            if 'discrete' in model_name:
                print("  Expected: Short=0.3, Medium=0.5, Long=0.7 (discrete bins)")
            elif 'fixed' in model_name:
                print("  Expected: All users = 0.5 (fixed fusion)")
            elif 'learnable' in model_name:
                print("  Expected: Learned during training (check model params)")
            elif 'continuous' in model_name:
                print("  Expected: Smooth function of sequence length")
            
            print("  ⚠️  Alpha statistics not saved (enable with track_alpha=True)")
            print()
    
    print("\n💡 Alpha interpretation:")
    print("   • α close to 0: More weight on GNN (collaborative)")
    print("   • α close to 1: More weight on SASRec (sequential)")
    print("   • α = 0.5: Equal balance")

## Step 10b: Alpha Statistics (Hybrid Models Only)

For hybrid models, check what fusion weights (alpha values) were used for different user groups.

## Step 11: Visualize Learning Curves

Plot training loss and validation NDCG over epochs

In [ ]:
import json
import matplotlib.pyplot as plt
import glob
import os

# Find all experiment results
result_folders = glob.glob('results/*_*')

if len(result_folders) > 0:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Plot 1: Training Loss
    for folder in result_folders:
        history_path = os.path.join(folder, 'history.json')
        if os.path.exists(history_path):
            try:
                with open(history_path, 'r') as f:
                    history = json.load(f)
                
                # Extract model name from folder
                parts = os.path.basename(folder).split('_')
                model_name = '_'.join(parts[:-2]) if len(parts) > 2 else parts[0]
                
                if 'train_loss' in history and history['train_loss']:
                    ax1.plot(history['train_loss'], label=model_name, marker='o', markersize=3, linewidth=2)
            except Exception as e:
                print(f"⚠️  Could not load history from {folder}: {e}")
    
    ax1.set_xlabel('Epoch', fontsize=12)
    ax1.set_ylabel('BPR Loss', fontsize=12)
    ax1.set_title('Training Loss', fontsize=14, fontweight='bold')
    ax1.legend(fontsize=10)
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Validation NDCG@10
    for folder in result_folders:
        history_path = os.path.join(folder, 'history.json')
        if os.path.exists(history_path):
            try:
                with open(history_path, 'r') as f:
                    history = json.load(f)
                
                parts = os.path.basename(folder).split('_')
                model_name = '_'.join(parts[:-2]) if len(parts) > 2 else parts[0]
                
                if 'val_metrics' in history and history['val_metrics']:
                    ndcg_values = [m.get('NDCG@10', 0) for m in history['val_metrics']]
                    if ndcg_values:
                        ax2.plot(ndcg_values, label=model_name, marker='o', markersize=3, linewidth=2)
            except Exception as e:
                print(f"⚠️  Could not load validation metrics from {folder}: {e}")
    
    ax2.set_xlabel('Epoch', fontsize=12)
    ax2.set_ylabel('NDCG@10', fontsize=12)
    ax2.set_title('Validation NDCG@10', fontsize=14, fontweight='bold')
    ax2.legend(fontsize=10)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    # Save plot
    os.makedirs('results', exist_ok=True)
    plt.savefig('results/learning_curves.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("✓ Saved to: results/learning_curves.png")
else:
    print("No results to plot. Run experiments first!")

## Step 12: Download Results

Create a zip file of all results for download

In [ ]:
# Create zip of all results
import os

if os.path.exists('results') and os.listdir('results'):
    !zip -r results.zip results/
    
    print("\n✅ Success!")
    print("Download 'results.zip' from the Output tab (right sidebar) →")
    print("\nContains:")
    print("  • Model checkpoints (best_model.pt)")
    print("  • Training history (history.json)")
    print("  • Test metrics (results.json)")
    print("  • Comparison tables (CSV files, if generated)")
    print("  • Learning curves (PNG)")
    
    # Show what's in results
    result_folders = [d for d in os.listdir('results') if os.path.isdir(os.path.join('results', d))]
    print(f"\n📦 Packaged {len(result_folders)} experiment(s):")
    for folder in result_folders:
        print(f"  • {folder}")
else:
    print("⚠️  No results folder found. Run experiments first!")

---

## ✅ Summary

You've successfully:
1. ✅ Cloned repository with preprocessed data
2. ✅ Installed all dependencies
3. ✅ Verified GPU availability
4. ✅ Tested training pipeline
5. ✅ Trained recommendation models
6. ✅ Analyzed and compared results
7. ✅ Visualized learning curves

---

## 🔬 Key Results

**Dataset:** MovieLens-1M
- 6,034 users
- 3,533 items  
- 1M+ ratings
- 151,874 co-occurrence edges

**Models Trained:**
- SASRec (Transformer baseline)
- Hybrid with Discrete Fusion (length-adaptive)

**Metrics:** Hit Rate (HR), NDCG, MRR at K={5, 10, 20}

---

## ⏱️ Training Time & Epochs FAQ

**Q: Why 50 epochs instead of 600 like in papers?**

**A:** We use **early stopping** (patience=10):
- Training automatically stops when validation NDCG@10 stops improving
- With 50 epochs max → usually converges at epoch 20-30 (~8-10 min GPU)
- With 600 epochs max → usually converges at epoch 30-40 (~35-45 min GPU)
- Performance difference: ~2-3% for 10x more training time

**Default (Fast):**
```bash
--epochs 50 --patience 10  # 8-10 min GPU, 95-98% of max performance
```

**Paper Setting (Thorough):**
```bash
--epochs 600 --patience 20  # 35-45 min GPU with early stopping, 100% performance
```

**Without Early Stopping (Not Recommended):**
```bash
--epochs 600 --patience 9999  # 80+ min GPU, risk of overfitting
```

---

## 🚀 Next Steps

**1. Match paper settings (600 epochs):**
```python
!python experiments/run_experiment.py \
    --model hybrid_discrete \
    --epochs 600 \
    --patience 20 \
    --batch_size 256 \
    --lr 0.001
```

**2. Experiment with hyperparameters:**
```python
!python experiments/run_experiment.py \
    --model hybrid_discrete \
    --epochs 100 \
    --batch_size 512 \
    --lr 0.0005 \
    --d_model 128 \
    --n_heads 4
```

**3. Try different fusion strategies:**
- `--model hybrid_learnable` - Per-user learned weights
- `--model hybrid_continuous` - Neural network fusion
- `--model hybrid_fixed` - Fixed alpha=0.5

**4. Analyze specific user groups:**
Check `results/comparison_*.csv` for performance on short/medium/long history users

---

## 📚 Resources

- **GitHub:** https://github.com/faroukq1/length-adaptive
- **Paper:** Length-Adaptive Hybrid Sequential Recommendation
- **Dataset:** MovieLens-1M (GroupLens)

---

**Questions or issues?** Check the README.md and EXPERIMENTS.md in the repository.